# Laboratorio 04 | The Games of Nodes Part II

Agregaciones en Neo4j (Cypher) usando Count, Collect y Size

## Configuración del entorno

In [19]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os

In [20]:
load_dotenv()

URI = os.getenv("NEO4J_URI")
AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))

driver = GraphDatabase.driver(URI, auth=AUTH)

def close_connection():
    driver.close()

def verify_connection():
    try:
        driver.verify_connectivity()
        print("Connection verified")
        with driver.session() as session:
            result = session.run("RETURN 1 AS value")
            value = result.single()["value"]
            print("Connection successful. Result:", value)
    except Exception as e:
        print("Connection failed:", e)

### Verificar la conexión

In [21]:
verify_connection()
# close_connection()

Connection verified
Connection successful. Result: 1


# 🔎 Acto 01: El Descenso de los Cineastas
Somos detectives de datos en NeoCity, donde cada persona, lugar y evento está conectado en un vasto grafo. Últimamente, la comunidad de cineastas ha visto una caída en el número de directores activos, y nos han contratado para averiguar por qué.

### **Investigación 1:** Determinar la cantidad de cineastas antes y después de 2010
- **Objetivo:** Determinar cuántos cineastas han ingresado a la industria antes y después del 2010.

In [22]:
def get_arround_2010():
  query1 = """
    MATCH (p:Person)
    WHERE p.yearJoined < 2010
    RETURN COUNT(p) AS CineastasAntes
  """
  query2 = """
    MATCH (p:Person)
    WHERE p.yearJoined >= 2010
    RETURN COUNT(p) AS CineastasDespues
  """
  with driver.session() as session:
    result1 = session.run(query1)
    cineastas_antes = result1.single()["CineastasAntes"]
    
    result2 = session.run(query2)
    cineastas_despues = result2.single()["CineastasDespues"]
    
    return cineastas_antes, cineastas_despues

cineastas_antes_2010, cineastas_despues_2010 = get_arround_2010()
print("Cineastas antes de 2010:", cineastas_antes_2010)
print("Cineastas después de 2010:", cineastas_despues_2010)

Cineastas antes de 2010: 6
Cineastas después de 2010: 4


>Pista revelada: Se ha determinado una notable disminución en la contratación de nuevos cineastas después de 2010.

### **Investigación 2:** Analizar los cambios en la industria con base en los géneros
- **Objetivo:** Investigar los cambios en la industria que podrían estar relacionados con los géneros de las películas. Uso de COUNT().

In [28]:
def get_movie_genres_count():
  query = """
  MATCH (m:Movie)-[:BELONGS_TO]->(g:Genre)
  RETURN g.name AS Genero, COUNT(m) AS NumPeliculas
  ORDER BY NumPeliculas DESC
  """
  with driver.session() as session:
    result = session.run(query)
    genres_count = [{"Genero": record["Genero"], "NumPeliculas": record["NumPeliculas"]} for record in result]
    return genres_count

movie_genres_count = get_movie_genres_count()

for genre in movie_genres_count:
    print(f"Genero: {genre['Genero']}, NumPeliculas: {genre['NumPeliculas']}")

Genero: Drama, NumPeliculas: 6
Genero: Acción, NumPeliculas: 6
Genero: Suspenso, NumPeliculas: 5
Genero: Ciencia Ficción, NumPeliculas: 4
Genero: Terror, NumPeliculas: 3
Genero: Comedia, NumPeliculas: 1


>Pista revelada: Se observa que algunos géneros, como el suspenso y la acción, han visto un aumento en producción, mientras que otros han decaído.

###  **Investigación 3:** ¿Cómo ha afectado la disminución de cineastas a las colaboraciones?
- Objetivo: Investigar si la disminución de cineastas ha afectado la colaboración dentro de la comunidad. Uso de COUNT().

In [34]:
def get_collaborations_count():
  query = """
  MATCH (p:Person)-[:COLLABORATED_WITH]->(colleague)
  RETURN COUNT(DISTINCT p) AS CineastasColaborando
  """
  with driver.session() as session:
    result = session.run(query)
    collaborations_count = result.single()["CineastasColaborando"]
    return collaborations_count

collaborations_count = get_collaborations_count()
print("Colaboraciones entre cineastas:", collaborations_count)

Colaboraciones entre cineastas: 9


>Pista revelada: Las colaboraciones han aumentado, lo que sugiere que la comunidad cinematográfica es más unida y competitiva que antes.

## **🎭 Acto 02: Los Vínculos Ocultos**
Con pistas sobre la **disminución de cineastas** y **cambios en la industria**, buscamos **profundizar en las conexiones** de los cineastas para entender mejor la situación.

### **Investigación 4:** Agrupar cineastas por año de unión

- Objetivo: Ver tendencias en la adhesión de nuevos cineastas a lo largo del tiempo.  

In [22]:
def get_cineastas_por_ano():
    query = """
    MATCH (p:Person)
    RETURN p.yearJoined AS AñoIngreso, COLLECT(p.name) AS Cineastas
    ORDER BY AñoIngreso
    """
    with driver.session() as session:
        result = session.run(query)
        cineastas_por_ano = [{"AñoIngreso": record["AñoIngreso"], "Cineastas": record["Cineastas"]} for record in result]
        return cineastas_por_ano

cineastas_por_ano = get_cineastas_por_ano()

for entry in cineastas_por_ano:
    print(f"Año de Ingreso: {entry['AñoIngreso']}, Cineastas: {', '.join(entry['Cineastas'])}")

Año de Ingreso: 1985, Cineastas: Roberto Díaz
Año de Ingreso: 1990, Cineastas: Juan Pérez
Año de Ingreso: 1998, Cineastas: Alejandro González
Año de Ingreso: 2000, Cineastas: Carlos Méndez
Año de Ingreso: 2003, Cineastas: Sofia Ramírez
Año de Ingreso: 2005, Cineastas: Laura Fernández
Año de Ingreso: 2010, Cineastas: Carla Mendoza
Año de Ingreso: 2012, Cineastas: Elena Gutiérrez
Año de Ingreso: 2015, Cineastas: Miguel Torres
Año de Ingreso: 2018, Cineastas: Valeria Soto


>**Pista revelada:** Se descubre que los cineastas más **antiguos están retirándose o cambiando de carrera**.

### **Investigación 5:** Identificar las películas dirigidas por cada cineasta

- Objetivo: Coleccionar los títulos de películas dirigidas por cada cineasta.  

In [23]:
def get_peliculas_dirigidas():
    query = """
    MATCH (p:Person)-[:DIRECTED]->(m:Movie)
    RETURN p.name AS Director, COLLECT(m.title) AS PeliculasDirigidas
    """
    with driver.session() as session:
        result = session.run(query)
        peliculas_dirigidas = [{"Director": record["Director"], "PeliculasDirigidas": record["PeliculasDirigidas"]} for record in result]
        return peliculas_dirigidas

peliculas_dirigidas = get_peliculas_dirigidas()

for entry in peliculas_dirigidas:
    print(f"Director: {entry['Director']}, Peliculas Dirigidas: {', '.join(entry['PeliculasDirigidas'])}")

Director: Alejandro González, Peliculas Dirigidas: El Último Acto, Destino Inesperado
Director: Sofia Ramírez, Peliculas Dirigidas: Ciudad Oscura, Sueños Rotos
Director: Juan Pérez, Peliculas Dirigidas: Fuego Interno, Horizonte Perdido, Más Allá del Miedo
Director: Carla Mendoza, Peliculas Dirigidas: Sin Retorno, El Último Refugio
Director: Miguel Torres, Peliculas Dirigidas: Caos Total, Código Secreto
Director: Laura Fernández, Peliculas Dirigidas: Fantasmas del Pasado, Caminos Cruzados
Director: Roberto Díaz, Peliculas Dirigidas: Guerra de Titanes, El Silencio Mortal, El Fin del Juego, Pánico en la Ciudad
Director: Elena Gutiérrez, Peliculas Dirigidas: Dimensión Desconocida, La Verdad Prohibida
Director: Carlos Méndez, Peliculas Dirigidas: El Espejo Roto, Lágrimas en el Viento, Colapso


>Pista revelada: Algunos directores clave han cambiado su enfoque a géneros más rentables, abandonando otros.


### **Investigación 6:** Agrupar películas por género y director

- Objetivo: Entender cómo los directores han diversificado su producción.  

In [24]:
def get_peliculas_por_genero_y_director():
    query = """
    MATCH (p:Person)-[:DIRECTED]->(m:Movie)-[:BELONGS_TO]->(g:Genre)
    RETURN p.name AS Director, g.name AS Genero, COLLECT(m.title) AS Peliculas
    ORDER BY Director
    """
    with driver.session() as session:
        result = session.run(query)
        peliculas_por_genero_y_director = [{"Director": record["Director"], "Genero": record["Genero"], "Peliculas": record["Peliculas"]} for record in result]
        return peliculas_por_genero_y_director

peliculas_por_genero_y_director = get_peliculas_por_genero_y_director()

for entry in peliculas_por_genero_y_director:
    print(f"Director: {entry['Director']}, Genero: {entry['Genero']}, Peliculas: {', '.join(entry['Peliculas'])}")

Director: Alejandro González, Genero: Drama, Peliculas: El Último Acto
Director: Alejandro González, Genero: Acción, Peliculas: Destino Inesperado
Director: Carla Mendoza, Genero: Suspenso, Peliculas: Sin Retorno
Director: Carla Mendoza, Genero: Drama, Peliculas: El Último Refugio
Director: Carlos Méndez, Genero: Terror, Peliculas: El Espejo Roto
Director: Carlos Méndez, Genero: Drama, Peliculas: Lágrimas en el Viento
Director: Carlos Méndez, Genero: Ciencia Ficción, Peliculas: Colapso
Director: Elena Gutiérrez, Genero: Ciencia Ficción, Peliculas: Dimensión Desconocida
Director: Elena Gutiérrez, Genero: Suspenso, Peliculas: La Verdad Prohibida
Director: Juan Pérez, Genero: Acción, Peliculas: Fuego Interno
Director: Juan Pérez, Genero: Ciencia Ficción, Peliculas: Horizonte Perdido
Director: Juan Pérez, Genero: Terror, Peliculas: Más Allá del Miedo
Director: Laura Fernández, Genero: Suspenso, Peliculas: Fantasmas del Pasado
Director: Laura Fernández, Genero: Drama, Peliculas: Caminos Cru

>**Pista revelada:** Se observa una **diversificación de géneros** en los cineastas más recientes, probablemente para atraer a una audiencia más amplia.